In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [0]:
ls

In [0]:
import os
os.chdir("/content/gdrive/My Drive/FER")

In [0]:
ls

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

# Preprocess Data

In [0]:
import pandas as pd
import csv
import numpy as np
import h5py
from PIL import Image

In [0]:
class PreTrainedResNet(nn.Module):
  def __init__(self, num_classes, feature_extracting):
    super(PreTrainedResNet, self).__init__()
    
    #TODO1: Load pre-trained ResNet Model
    self.resnet18 = models.resnet18(pretrained=True)

    #Set gradients to false
    if feature_extracting:
      for param in self.resnet18.parameters():
          param.requires_grad = False
    
    #Replace last fc layer
    num_feats = self.resnet18.fc.in_features
    
    #TODO2: Replace fc layer in resnet to a linear layer of size (num_feats, num_classes)
    self.resnet18.fc = torch.nn.Linear(num_feats, num_classes)
    
  def forward(self, x):
    #TODO3: Forward pass x through the model
    x = self.resnet18(x)
    return x

In [0]:
import os
import cv2
from os import listdir
import numpy as np
from os.path import isfile, join
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [0]:
PATH = 'training/'
emotion_list = ['anger', 'disgust', 'fear', 'happy', 'sadness', 'surprise', 'contempt']
img_list = []
train_lab = []

count = 0
ct = 0
for emotion in emotion_list:
  mypath = PATH + '/' + emotion
  files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
  for file in files:
    file_path = mypath + '/' + file
    img = cv2.imread(file_path, 0)
    #print(img.shape)
    ct += 1
    #print(ct)
    img = np.repeat(img, 3)
    img = img.reshape((48, 48, 3))
    img_list.append(img)
    train_lab.append(count)
  count += 1

img_arr = np.array(img_list)
#print(img_arr.shape)
train_val = img_arr
train_lab = np.array(train_lab)
#print(test_lab.shape)

PATH = 'testing/'
emotion_list = ['anger', 'disgust', 'fear', 'happy', 'sadness', 'surprise', 'contempt']
img_list = []
test_lab = []

count = 0
ct = 0
for emotion in emotion_list:
  mypath = PATH + '/' + emotion
  files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
  for file in files:
    file_path = mypath + '/' + file
    img = cv2.imread(file_path, 0)
    #print(img.shape)
    ct += 1
    #print(ct)
    img = np.repeat(img, 3)
    img = img.reshape((48, 48, 3))
    img_list.append(img)
    test_lab.append(count)
  count += 1

img_arr = np.array(img_list)
#print(img_arr.shape)
test_val = img_arr
test_lab = np.array(test_lab)
#print(test_lab.shape)

In [0]:
test_lab.shape[0]

In [0]:
transformations = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),])

trainset = CK(transform=transformations)
trainset_loader = torch.utils.data.DataLoader(dataset=trainset,
                                                    batch_size=8,
                                                    shuffle=True, num_workers=4)

testset = CK(transform=transformations)
testset_loader = torch.utils.data.DataLoader(dataset=testset,
                                                    batch_size=8,
                                                    shuffle=True, num_workers=4)

In [0]:
class CK(torch.utils.data.Dataset):
  def __init__(self, transform=None):
    self.to_tensor = transforms.ToTensor()
    self.transform = transform
    self.train_data = train_val
    self.train_label = train_lab
    
  def __getitem__(self, index):
    train_instance = self.train_data[index]
    train_img = Image.fromarray(train_instance)
    train_img = train_img.convert('RGB')
    if self.transform is not None:
      train_tensor = self.transform(train_img)
    return (train_tensor, self.train_label[index])
    
  def __len__(self):
    return len(self.train_label)

In [0]:
def train(model, optimizer, criterion, epoch, num_epochs):
  model.train()
  epoch_loss = 0.0
  epoch_acc = 0.0
  
  for batch_idx, (images, labels) in enumerate(trainset_loader):
    #zero the parameter gradients
    optimizer.zero_grad()
    
    #move to GPU
    images, labels = images.cuda(), labels.cuda()
    
    #forward
    outputs = model.forward(images)
#     print(labels)
    
    loss = criterion(outputs, labels)
#     print("loss: " + str(loss))
    
    _, preds = torch.max(outputs.data, 1)
    
    loss.backward()
    optimizer.step()
    
    epoch_loss += loss.item()
    epoch_acc += torch.sum(preds == labels).item()
    
  epoch_loss /= train_lab.shape[0]
  epoch_acc /= train_lab.shape[0]
  
  print('TRAINING Epoch %d/%d Loss %.4f Accuracy %.4f' % (epoch, num_epochs, epoch_loss, epoch_acc))

# for i, (images, labels) in enumerate(dataset_loader):
#   print(len(images))

In [0]:
NUM_EPOCHS = 15
LEARNING_RATE = 0.01
BATCH_SIZE = 8
RESNET_LAST_ONLY = False #Fine tunes only the last layer. Set to False to fine tune entire network

#Initialize the model
CK_model = PreTrainedResNet(7, RESNET_LAST_ONLY)
CK_model = CK_model.cuda()

#Setting the optimizer and loss criterion
optimizer = optim.SGD(CK_model.parameters(), lr=LEARNING_RATE, momentum=0.9)
criterion = nn.CrossEntropyLoss()

#Begin Train
for epoch in range(NUM_EPOCHS):
  train(CK_model, optimizer, criterion, epoch+1, NUM_EPOCHS)
  
print("Finished Training")
print("-"*10)

In [0]:
def test(model, criterion, repeats=2):
  model.eval()
  
  test_loss = 0.0
  test_acc = 0.0
  
  label_arr = []
  pred_arr = []
  
  with torch.no_grad():
    for itr in range(repeats):
      for batch_idx, (images, labels) in enumerate(testset_loader):
        #move to GPU
        
        images, labels = images.cuda(), labels.cuda()

        #forward
        if itr==1:
          label = np.array(labels.cpu())
          label_arr.extend(label)
          
        outputs = model.forward(images)

        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs.data, 1)
        
        if itr==1:
          pred = np.array(preds.cpu())
          pred_arr.extend(pred)

        test_loss += loss.item()
        test_acc += torch.sum(preds == labels).item()

    test_loss /= (test_lab.shape[0]*repeats)
    test_acc /= (test_lab.shape[0]*repeats)

    print('Test Loss: %.4f Test Accuracy %.4f' % (test_loss, test_acc))
    
  return label_arr, pred_arr

In [0]:
lab, pre = test(CK_model,criterion, repeats=2)

In [0]:
confusion_matrix(lab, pre)


In [0]:
test(model, criterion, repeats = 2)

In [0]:
!pip3 install face_recognition

In [0]:
import face_recognition
import argparse
import pickle
import cv2
import numpy as np
import scipy.misc
from google.colab.patches import cv2_imshow

In [0]:
def detect(image):
    if image.shape[2] != 3:
        print('...')


    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    boxes = face_recognition.face_locations(rgb,
    	model = 'cnn')
    print(boxes)
    images = []
    for box in boxes:
        img = image[box[0] : box[2] , box[3] : box[1]]
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        images.append(gray)
        
    for image in images:
      print(image)
      cv2_imshow(image)
        
    new_images = []
    for image in images:
      shape = np.shape(image)
      new_image = np.repeat(image, 3)
      new_image = new_image.reshape((shape[0], shape[1], 3)).astype('uint8')
      new_images.append(new_image)

    return new_images, boxes

In [0]:
test_img = cv2.imread('test.jpg')
faces, boxes = np.array(detect(test_img))
print(np.shape(faces[0]))

In [0]:
class test_image(torch.utils.data.Dataset):
  def __init__(self, transform=None):
    self.to_tensor = transforms.ToTensor()
    self.transform = transform
    test_img = cv2.imread('test.jpg')
    faces, boxes = np.array(detect(test_img))
    self.data = faces
    
  def __getitem__(self, index):
    instance = self.data[index]
    img = Image.fromarray(instance)
    img = img.convert('RGB')
    if self.transform is not None:
      instance_tensor = self.transform(img)
    return instance_tensor
    
  def __len__(self):
    return len(self.data)

In [0]:
labels = []
with torch.no_grad():
  for batch_idx, images in enumerate(dataset_loader):
#   outputs = model.forward(images)
#   print(outputs)
    print(np.shape(np.array(images)))
    images = images.cuda()
    outputs = model.forward(images)
    _, preds = torch.max(outputs.data, 1)
    pred1 = preds.cpu()
    print(np.array(pred1))
    labels.extend(np.array(pred1))
print(labels)

In [0]:
class_list = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
face_list = []
for ((top, right, bottom, left), label) in zip(boxes, labels):
  cv2.rectangle(test_img, (left, top), (right, bottom), (0, 255, 0), 2)
  y = top - 15 if top - 15 > 15 else top + 15
  cv2.putText(test_img, class_list[label], (left, y), cv2.FONT_HERSHEY_SIMPLEX,
            0.75, (0, 255, 0), 2)
  face_list.append((label, ((top+bottom)/2, (left+right)/2)))

print(face_list)
# cv2.namedWindow("output", cv2.WINDOW_NORMAL)
im2 = test_img.copy()
# im2[:,:,0] = test_img[:,:,2]
# im2[:, :, 2] = test_img[:, :, 0]
scipy.misc.imsave('kkk.jpg', im2)
cv2_imshow(im2)